Complete the following tasks with Python Boto code
1. Create ECS Cluster with 1 Container Instance
2. Create a Task Definition (apache and wordpress)
3. Create an ELB and Target Group to later associate with the ECS Service
4. Create a Service that runs the Task Definition
5. Confirm Everything is Working
6. Scale Up the Service to 4 Tasks.
7. Clean It All Up

Docs:
https://docs.aws.amazon.com/AmazonECS/latest/developerguide/launch_container_instance.html

In [1]:
import boto3
import pprint
import time

ecs_client = boto3.client('ecs')

In [18]:
mycluster = 'myTestCluster'

createClusterResponse = ecs_client.create_cluster(
                        
                        clusterName = mycluster,

                        )



In [19]:
pprint.pprint(createClusterResponse)

{'ResponseMetadata': {'HTTPHeaders': {'content-length': '255',
                                      'content-type': 'application/x-amz-json-1.1',
                                      'date': 'Sun, 30 Sep 2018 02:44:17 GMT',
                                      'x-amzn-requestid': 'b96f4db9-c45a-11e8-a953-f388d3d45831'},
                      'HTTPStatusCode': 200,
                      'RequestId': 'b96f4db9-c45a-11e8-a953-f388d3d45831',
                      'RetryAttempts': 0},
 'cluster': {'activeServicesCount': 0,
             'clusterArn': 'arn:aws:ecs:us-east-1:117198229476:cluster/myTestCluster',
             'clusterName': 'myTestCluster',
             'pendingTasksCount': 0,
             'registeredContainerInstancesCount': 4,
             'runningTasksCount': 0,
             'statistics': [],
             'status': 'ACTIVE'}}


In [2]:
ec2_client = boto3.client('ec2')

user_data_script ="""#!/bin/bash
echo ECS_CLUSTER=myTestCluster >> /etc/ecs/ecs.config"""

ec2Response = ec2_client.run_instances(

                ImageId = 'ami-0b9a214f40c38d5eb',
#                 ImageId = 'ami-00129b193dc81bc31',
                MinCount = 1,
                MaxCount = 4,
                InstanceType = 't2.micro',
                KeyName='ec2-1',
                SecurityGroupIds=['sg-0fffb748eaf870000'],
                IamInstanceProfile = {
                    "Name" : "ecsInstanceRole"
                },
                UserData = user_data_script,
                SubnetId='subnet-3715906b',

            )

In [3]:
pprint.pprint(ec2Response)

{'Groups': [],
 'Instances': [{'AmiLaunchIndex': 1,
                'Architecture': 'x86_64',
                'BlockDeviceMappings': [],
                'ClientToken': '',
                'CpuOptions': {'CoreCount': 1, 'ThreadsPerCore': 1},
                'EbsOptimized': False,
                'Hypervisor': 'xen',
                'IamInstanceProfile': {'Arn': 'arn:aws:iam::117198229476:instance-profile/ecsInstanceRole',
                                       'Id': 'AIPAJPWEQQZCTGQDY5WVO'},
                'ImageId': 'ami-0b9a214f40c38d5eb',
                'InstanceId': 'i-02f0d898ad1cb9921',
                'InstanceType': 't2.micro',
                'KeyName': 'ec2-1',
                'LaunchTime': datetime.datetime(2018, 9, 30, 2, 11, 2, tzinfo=tzutc()),
                'Monitoring': {'State': 'disabled'},
                'NetworkInterfaces': [{'Attachment': {'AttachTime': datetime.datetime(2018, 9, 30, 2, 11, 2, tzinfo=tzutc()),
                                                    

In [4]:
# pprint.pprint(ec2Response['Instances'][1]['InstanceId'])
ids =[]
for i in ec2Response['Instances']:
    ids.append(i['InstanceId'])
    
print(ids)

['i-02f0d898ad1cb9921', 'i-0267a8c8eda3f830e', 'i-0f19c19b7790ee151', 'i-04d2f036ff15bd654']


In [10]:
mytargetgroup = 'myTG'
tg_response = boto3.client('elbv2').create_target_group(
    Name=mytargetgroup,
    Port=80,
    Protocol='HTTP',
    VpcId='vpc-c54c99bf',
)

elbname = 'myElb'
elb_response = boto3.client('elbv2').create_load_balancer(

                Name = elbname,
#                 Listeners = [
#                   {
#                     'InstancePort': 8080,
#                     'InstanceProtocol': 'HTTP',
#                     'LoadBalancerPort': 80,
#                     'Protocol': 'HTTP',
#                   },      
#                 ],
                Subnets=[
                    'subnet-3715906b',
                    'subnet-eb770ae4',
                    'subnet-9bd59dd1',
                    'subnet-9a088fb4',
                    'subnet-8d47ffea',
                    'subnet-66b67958',
                    'subnet-3715906b',
                ],
                SecurityGroups=[
                    'sg-0fffb748eaf870000',
                ],
                )

pprint.pprint(tg_response)
pprint.pprint(elb_response)


{'ResponseMetadata': {'HTTPHeaders': {'content-length': '1148',
                                      'content-type': 'text/xml',
                                      'date': 'Sun, 30 Sep 2018 02:35:53 GMT',
                                      'x-amzn-requestid': '8d70718b-c459-11e8-a0c0-d7985b799376'},
                      'HTTPStatusCode': 200,
                      'RequestId': '8d70718b-c459-11e8-a0c0-d7985b799376',
                      'RetryAttempts': 0},
 'TargetGroups': [{'HealthCheckIntervalSeconds': 30,
                   'HealthCheckPath': '/',
                   'HealthCheckPort': 'traffic-port',
                   'HealthCheckProtocol': 'HTTP',
                   'HealthCheckTimeoutSeconds': 5,
                   'HealthyThresholdCount': 5,
                   'Matcher': {'HttpCode': '200'},
                   'Port': 80,
                   'Protocol': 'HTTP',
                   'TargetGroupArn': 'arn:aws:elasticloadbalancing:us-east-1:117198229476:targetgroup/myTG/2be

In [13]:
# pprint.pprint(elb_response)
tgarn = tg_response['TargetGroups'][0]['TargetGroupArn']
lbarn = elb_response['LoadBalancers'][0]['LoadBalancerArn']

In [14]:
listner_response =  boto3.client('elbv2').create_listener(
    DefaultActions=[
        {
            'TargetGroupArn': tgarn,
            'Type': 'forward',
        },
    ],
    LoadBalancerArn=lbarn,
    Port=80,
    Protocol='HTTP',
)


NameError: name 'listener_response' is not defined

In [15]:
pprint.pprint(listner_response)

{'Listeners': [{'DefaultActions': [{'TargetGroupArn': 'arn:aws:elasticloadbalancing:us-east-1:117198229476:targetgroup/myTG/2be330f7376a72a8',
                                    'Type': 'forward'}],
                'ListenerArn': 'arn:aws:elasticloadbalancing:us-east-1:117198229476:listener/app/myElb/d7bf58f282bfd8d7/4928e4148d09ff48',
                'LoadBalancerArn': 'arn:aws:elasticloadbalancing:us-east-1:117198229476:loadbalancer/app/myElb/d7bf58f282bfd8d7',
                'Port': 80,
                'Protocol': 'HTTP'}],
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '926',
                                      'content-type': 'text/xml',
                                      'date': 'Sun, 30 Sep 2018 02:39:01 GMT',
                                      'x-amzn-requestid': 'fd0cd461-c459-11e8-af6c-ab015d2d5d45'},
                      'HTTPStatusCode': 200,
                      'RequestId': 'fd0cd461-c459-11e8-af6c-ab015d2d5d45',
                      'RetryAttempts':

In [16]:
task_response = ecs_client.register_task_definition(
    
        containerDefinitions= [
        {
            "environment": [
                {
                    "name": "WORDPRESS_DB_PASSWORD",
                    "value": "ChangeMeIfYouWant"
                }
            ],
            "essential": True,
            "image": "wordpress",
            "memory": 300,
            "cpu": 10,
            "links": [
                "my-wpdb:mysql"
            ],
#             "mountPoints": [
#                 {
#                     "containerPath": "/var/www/html",
#                     "sourceVolume": "vol-012b57b135a18d576"
#                 }
#             ],
            "name": "my-wp",
            "portMappings": [
                {
                    "containerPort": 80,
                    "hostPort": 8080
                }
            ]
        },
        {
            "environment": [
                {
                    "name": "MYSQL_ROOT_PASSWORD",
                    "value": "ChangeMeIfYouWant"
                }
            ],
            "essential": True,
            "image": "mariadb",
            "name": "my-wpdb",
            "memory": 300,
            "cpu": 10,
            "portMappings": [
                {
                    "containerPort": 3306,
                    "hostPort": 8081
                }
            ]
        }
    ],

#     volumes= [
#         {
#             "host": {
#                 "sourcePath": "./"
#             },
#             "name": "vol-012b57b135a18d576"
#         }
#     ],
        family="workffs"
    )


pprint.pprint(task_response)

task_arn = task_response['taskDefinition']['taskDefinitionArn']

{'ResponseMetadata': {'HTTPHeaders': {'content-length': '785',
                                      'content-type': 'application/x-amz-json-1.1',
                                      'date': 'Sun, 30 Sep 2018 02:43:51 GMT',
                                      'x-amzn-requestid': 'aa365bfb-c45a-11e8-9c59-5f0ff0209fc8'},
                      'HTTPStatusCode': 200,
                      'RequestId': 'aa365bfb-c45a-11e8-9c59-5f0ff0209fc8',
                      'RetryAttempts': 0},
 'taskDefinition': {'compatibilities': ['EC2'],
                    'containerDefinitions': [{'cpu': 10,
                                              'environment': [{'name': 'WORDPRESS_DB_PASSWORD',
                                                               'value': 'ChangeMeIfYouWant'}],
                                              'essential': True,
                                              'image': 'wordpress',
                                              'links': ['my-wpdb:mysql'],
         

In [20]:
myservice = 'wordpress_service'

create_service_response = ecs_client.create_service(
                
                            cluster = mycluster,
                            serviceName= myservice,
                            taskDefinition= task_arn,
                            desiredCount=1,
                            clientToken= 'my_token',
                            launchType='EC2',
                            loadBalancers=[
                              {
                                'targetGroupArn': tgarn,
#                                 'loadBalancerName':elbname,
                                'containerPort': 80,
                                'containerName':'my-wp'
                              },  
                            ],
                            healthCheckGracePeriodSeconds=500,
                        )

pprint.pprint(create_service_response)

{'ResponseMetadata': {'HTTPHeaders': {'content-length': '1167',
                                      'content-type': 'application/x-amz-json-1.1',
                                      'date': 'Sun, 30 Sep 2018 02:44:55 GMT',
                                      'x-amzn-requestid': 'd09ad9fa-c45a-11e8-abaf-6bd0efc66c68'},
                      'HTTPStatusCode': 200,
                      'RequestId': 'd09ad9fa-c45a-11e8-abaf-6bd0efc66c68',
                      'RetryAttempts': 0},
 'service': {'clusterArn': 'arn:aws:ecs:us-east-1:117198229476:cluster/myTestCluster',
             'createdAt': datetime.datetime(2018, 9, 30, 8, 14, 56, 232000, tzinfo=tzlocal()),
             'deploymentConfiguration': {'maximumPercent': 200,
                                         'minimumHealthyPercent': 100},
             'deployments': [{'createdAt': datetime.datetime(2018, 9, 30, 8, 14, 56, 232000, tzinfo=tzlocal()),
                              'desiredCount': 1,
                              'i

In [21]:
boto3.client('ecs').update_service(
    cluster=mycluster,
    service=myservice,
    desiredCount=4,
)

{'service': {'serviceArn': 'arn:aws:ecs:us-east-1:117198229476:service/wordpress_service',
  'serviceName': 'wordpress_service',
  'clusterArn': 'arn:aws:ecs:us-east-1:117198229476:cluster/myTestCluster',
  'loadBalancers': [{'targetGroupArn': 'arn:aws:elasticloadbalancing:us-east-1:117198229476:targetgroup/myTG/2be330f7376a72a8',
    'containerName': 'my-wp',
    'containerPort': 80}],
  'serviceRegistries': [],
  'status': 'ACTIVE',
  'desiredCount': 4,
  'runningCount': 1,
  'pendingCount': 0,
  'launchType': 'EC2',
  'taskDefinition': 'arn:aws:ecs:us-east-1:117198229476:task-definition/workffs:4',
  'deploymentConfiguration': {'maximumPercent': 200,
   'minimumHealthyPercent': 100},
  'deployments': [{'id': 'ecs-svc/9223370498579279568',
    'status': 'PRIMARY',
    'taskDefinition': 'arn:aws:ecs:us-east-1:117198229476:task-definition/workffs:4',
    'desiredCount': 4,
    'pendingCount': 0,
    'runningCount': 1,
    'createdAt': datetime.datetime(2018, 9, 30, 8, 14, 56, 232000, t

In [24]:
boto3.client('ecs').update_service(
    cluster=mycluster,
    service=myservice,
    desiredCount=0,
)
boto3.client('ecs').delete_service(
    cluster=mycluster,
    service=myservice,
    force=True,
)

boto3.client('elbv2').delete_load_balancer(
    LoadBalancerArn = lbarn,
)
boto3.client('elbv2').delete_target_group(
    TargetGroupArn=tgarn,
)


{'ResponseMetadata': {'RequestId': 'fab473b8-c45e-11e8-9fa2-5fd0dc1cb713',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'fab473b8-c45e-11e8-9fa2-5fd0dc1cb713',
   'content-type': 'text/xml',
   'content-length': '259',
   'date': 'Sun, 30 Sep 2018 03:14:43 GMT'},
  'RetryAttempts': 0}}

In [25]:
boto3.resource('ec2').instances.filter(InstanceIds=ids).stop()
boto3.resource('ec2').instances.filter(InstanceIds=ids).terminate()

[{'TerminatingInstances': [{'CurrentState': {'Code': 32,
     'Name': 'shutting-down'},
    'InstanceId': 'i-02f0d898ad1cb9921',
    'PreviousState': {'Code': 64, 'Name': 'stopping'}},
   {'CurrentState': {'Code': 32, 'Name': 'shutting-down'},
    'InstanceId': 'i-04d2f036ff15bd654',
    'PreviousState': {'Code': 64, 'Name': 'stopping'}},
   {'CurrentState': {'Code': 32, 'Name': 'shutting-down'},
    'InstanceId': 'i-0f19c19b7790ee151',
    'PreviousState': {'Code': 64, 'Name': 'stopping'}},
   {'CurrentState': {'Code': 32, 'Name': 'shutting-down'},
    'InstanceId': 'i-0267a8c8eda3f830e',
    'PreviousState': {'Code': 64, 'Name': 'stopping'}}],
  'ResponseMetadata': {'RequestId': 'd534b5de-9a47-4d97-91f8-b6c9f3f8dd81',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'content-type': 'text/xml;charset=UTF-8',
    'transfer-encoding': 'chunked',
    'vary': 'Accept-Encoding',
    'date': 'Sun, 30 Sep 2018 03:14:53 GMT',
    'server': 'AmazonEC2'},
   'RetryAttempts': 0}}]

In [ ]:
time.sleep(30)
boto3.client('ecs').delete_cluster(
    cluster = mycluster,
)